## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/DSL/2022-2/정규세션/[0830] Text Mining & Word Embedding

/content/drive/MyDrive/DSL/2022-2/정규세션/[0830] Text Mining & Word Embedding


In [5]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,390 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.

In [3]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [6]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [7]:
os.getcwd()

'/content/drive/MyDrive/DSL/2022-2/정규세션/[0830] Text Mining & Word Embedding'

In [8]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [9]:
corpus[:50]

['분자를 부품처럼 활용한 세상서 가장 작은 기계',
 '메모리 소자 기술에 응용',
 '올해 노벨화학상은 일상 세계의 기계 장치를 분자 세계에서 구현해 세상에서 가장 작은 기계를 만든 과학자들에게 돌아갔다',
 '스웨덴 왕립과학원 노벨위원회는 5일 장피에르 소바주 프랑스 스트라부르대 교수(72)와 프레이저 스토더트 미국 노스웨스턴대 교수(74), 베르나르트 페링하 네덜란드 흐로닝언대 교수(65) 등 세 사람을 올해 노벨화학상 수상자로 선정했다고 발표했다',
 '위원회는 이들이 에너지를 운동으로 변환하는 기계장치를 분자 크기에서 구현해 화학과 기계 분야의 새 지평을 마련했다고 평가했다',
 '이들은 원자의 덩어리인 분자를 화학 반응이 아니라 기계처럼 사용하는 방법을 탐구했다',
 '여기에는 초(超)분자로 불리는 분자보다는 조금 크고 나노물질보다는 조금 작은 탄소 분자를 활용했다',
 '이들은 탄소원자를 기계적으로 이어붙여 고리를 만든 뒤 둘을 이어붙여 체인을 제작하거나 분자 막대 위에서 고리 하나가 왔다갔다하는 기계 장치로 구현했다',
 '프랑스 출신인 소바주 교수는 1983년 탄소원자로 만든 고리 두 개를 이어붙여 카테난이라는 초기 분자 기계를 개발했다',
 '영국 출신인 스토더트 교수는 1991년 분자 막대 하나와 분자 고리 하나를 연결시킨 로택산이란 분자 기계로 한 단계 발전시켰다',
 '네덜란드 출신인 페링하 교수는 1999년 세계에서 가장 작은 분자 모터를 개발하고 가장 작은 나노 자동차를 제작했다',
 '김기문 포스텍 화학과 교수는 “카테난과 로택산은 스위치 기능을 할 수 있어 집적도의 한계에 이른 메모리 소자를 대체할 기술로 활용하거나 약물 전달 물질의 밸브로 이용하는 연구가 진행되고 있다”고 설명했다',
 '노벨화학상 수상자 세 사람은 800만스웨덴크로나(약 10억3900만원)의 상금을 똑같이 나눠 갖는다',
 '접는 스마트폰 개발 전쟁이 치열하게 붙었다',
 '삼성전자가 지난 4월 접는 스마트폰의 특허를 특허청에 출원했다고 독일의 온라인 매체 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

=> 처음에는 OKT를 이용하여 문장에 대해 품사 태깅을 진행한 후 단어의 어간을 추출하고 정규화를 진행하였습니다. 그리고 품사가 명사이거나 동사인 단어들을 token list에 추가하였습니다. 이 경우, 하다, 되다 같은 크게 유의미하지 않은 토큰이 리스트에 추가되었음을 알 수 있었고, 뉴스 기사 특성상 명사만으로도 문장의 핵심적인 내용을 뽑아낼 수 있을거라고 생각하여 명사만을 추출하여 toekn list에 추가하였습니다.

In [10]:
okt = Okt()

In [11]:
ex1 = '여기에는 초(超)분자로 불리는 분자보다는 조금 크고 나노물질보다는 조금 작은 탄소 분자를 활용했다.'
ex2 = '식물에서 뿌리는 사람으로 치자면 \'뇌\'의 기능을 담당하는 것으로 드러났습니다'

# 형태소 분석
print(okt.morphs(ex1))

# 품사 태깅
print(okt.pos(ex1))

# 명사 추출
print(okt.nouns(ex1))

# 문장 정규화
print(okt.normalize(ex1))

['여기', '에는', '초', '(', '超', ')', '분자', '로', '불리는', '분자', '보다는', '조금', '크고', '나노', '물질', '보다는', '조금', '작은', '탄소', '분자', '를', '활용', '했다', '.']
[('여기', 'Noun'), ('에는', 'Josa'), ('초', 'Noun'), ('(', 'Punctuation'), ('超', 'Foreign'), (')', 'Punctuation'), ('분자', 'Noun'), ('로', 'Josa'), ('불리는', 'Verb'), ('분자', 'Noun'), ('보다는', 'Josa'), ('조금', 'Noun'), ('크고', 'Verb'), ('나노', 'Noun'), ('물질', 'Noun'), ('보다는', 'Josa'), ('조금', 'Noun'), ('작은', 'Adjective'), ('탄소', 'Noun'), ('분자', 'Noun'), ('를', 'Josa'), ('활용', 'Noun'), ('했다', 'Verb'), ('.', 'Punctuation')]
['여기', '초', '분자', '분자', '조금', '나노', '물질', '조금', '탄소', '분자', '활용']
여기에는 초(超)분자로 불리는 분자보다는 조금 크고 나노물질보다는 조금 작은 탄소 분자를 활용했다.


In [12]:
## stem/norm 비교
# stem = False, norm = False
print(okt.pos(ex1))

# stem = True, norm = False
print(okt.pos(ex1, stem = True))

# stem = False, norm = True
print(okt.pos(ex1, norm = True))

# stem = True, norm = True
print(okt.pos(ex1, stem = True, norm = True))

[('여기', 'Noun'), ('에는', 'Josa'), ('초', 'Noun'), ('(', 'Punctuation'), ('超', 'Foreign'), (')', 'Punctuation'), ('분자', 'Noun'), ('로', 'Josa'), ('불리는', 'Verb'), ('분자', 'Noun'), ('보다는', 'Josa'), ('조금', 'Noun'), ('크고', 'Verb'), ('나노', 'Noun'), ('물질', 'Noun'), ('보다는', 'Josa'), ('조금', 'Noun'), ('작은', 'Adjective'), ('탄소', 'Noun'), ('분자', 'Noun'), ('를', 'Josa'), ('활용', 'Noun'), ('했다', 'Verb'), ('.', 'Punctuation')]
[('여기', 'Noun'), ('에는', 'Josa'), ('초', 'Noun'), ('(', 'Punctuation'), ('超', 'Foreign'), (')', 'Punctuation'), ('분자', 'Noun'), ('로', 'Josa'), ('불리다', 'Verb'), ('분자', 'Noun'), ('보다는', 'Josa'), ('조금', 'Noun'), ('크다', 'Verb'), ('나노', 'Noun'), ('물질', 'Noun'), ('보다는', 'Josa'), ('조금', 'Noun'), ('작다', 'Adjective'), ('탄소', 'Noun'), ('분자', 'Noun'), ('를', 'Josa'), ('활용', 'Noun'), ('하다', 'Verb'), ('.', 'Punctuation')]
[('여기', 'Noun'), ('에는', 'Josa'), ('초', 'Noun'), ('(', 'Punctuation'), ('超', 'Foreign'), (')', 'Punctuation'), ('분자', 'Noun'), ('로', 'Josa'), ('불리는', 'Verb'), ('분자', 'Noun'), ('보다는'

In [13]:
pos_tagging = okt.pos(ex1, stem = True, norm = True)
for i in pos_tagging:
  if i[1] == 'Noun' or i[1] == 'Verb':
    print(i[0])

여기
초
분자
불리다
분자
조금
크다
나노
물질
조금
탄소
분자
활용
하다


In [14]:
pos_tagging = okt.pos(ex2, stem = True, norm = True)
print(pos_tagging)
for i in pos_tagging:
  if i[1] == 'Noun' or i[1] == 'Verb':
    print(i[0])

[('식물', 'Noun'), ('에서', 'Josa'), ('뿌리다', 'Verb'), ('사람', 'Noun'), ('으로', 'Josa'), ('치자', 'Noun'), ('면', 'Josa'), ("'", 'Punctuation'), ('뇌', 'Noun'), ("'", 'Punctuation'), ('의', 'Noun'), ('기능', 'Noun'), ('을', 'Josa'), ('담당', 'Noun'), ('하다', 'Verb'), ('것', 'Noun'), ('으로', 'Josa'), ('드러나다', 'Verb')]
식물
뿌리다
사람
치자
뇌
의
기능
담당
하다
것
드러나다


In [15]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  pos_tagging = okt.pos(sentence, stem = True, norm = True)
  tokens = []
  for i in pos_tagging:
    if i[1] == 'Noun' or i[1] == 'Verb':
      tokens.append(i[0])
  return tokens

In [16]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [17]:
print(corpus_tokenized)

[['분자', '부품', '활용', '세상', '가장', '기계'], ['메모리', '소자', '기술', '응용'], ['올해', '노벨화학상', '일상', '세계', '기계', '장치', '분자', '세계', '구현', '하다', '세상', '가장', '기계', '만들다', '과학자', '돌아가다'], ['스웨덴', '왕립', '학원', '노벨', '위원회', '장', '피에르', '소바', '주', '프랑스', '스트', '부르다', '교수', '오다', '프레이저', '스토', '더트', '미국', '노스웨스턴대', '교수', '베르나르', '트', '페링하', '네덜란드', '흐로닝언', '교수', '등', '세', '사람', '올해', '노벨화학상', '수상자', '선정', '하다', '발표', '하다'], ['위원회', '이', '에너지', '운동', '변환', '하다', '기계', '장치', '분자', '크기', '구현', '하다', '화학', '기계', '분야', '새', '지평', '마련', '하다', '평가', '하다'], ['이', '원자', '덩어리', '분자', '화학', '반응', '기계', '사용', '하다', '방법', '탐구', '하다'], ['여기', '초', '분자', '불리다', '분자', '조금', '크다', '나노', '물질', '조금', '탄소', '분자', '활용', '하다'], ['이', '탄소', '원자', '기계', '이다', '붙이다', '고리', '만들다', '뒤', '둘', '이다', '붙이다', '체인', '제작', '거나', '분자', '막대', '위', '고리', '하나', '오다', '가다', '기계', '장치', '구현', '하다'], ['프랑스', '출신', '소바', '주', '교수', '탄소', '원자로', '만들다', '고리', '두', '개', '이다', '붙이다', '크다', '난이', '초기', '분자', '기계', '개발', '하다'], ['영국', '출신', '스토', '더트', '

In [55]:
def CustomTokenization(sentence):
  noun_tokens = okt.nouns(sentence)
  tokens = []
  for i in noun_tokens:
    if len(i) != 1:
      tokens.append(i)
  return tokens

In [56]:
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [57]:
print(corpus_tokenized)

[['분자', '부품', '활용', '세상', '가장', '기계'], ['메모리', '소자', '기술', '응용'], ['올해', '노벨화학상', '일상', '세계', '기계', '장치', '분자', '세계', '구현', '세상', '가장', '기계', '과학자'], ['스웨덴', '왕립', '학원', '노벨', '위원회', '피에르', '소바', '프랑스', '스트', '교수', '프레이저', '스토', '더트', '미국', '노스웨스턴대', '교수', '베르나르', '페링하', '네덜란드', '흐로닝언', '교수', '사람', '올해', '노벨화학상', '수상자', '선정', '발표'], ['위원회', '에너지', '운동', '변환', '기계', '장치', '분자', '크기', '구현', '화학', '기계', '분야', '지평', '마련', '평가'], ['원자', '덩어리', '분자', '화학', '반응', '기계', '사용', '방법', '탐구'], ['여기', '분자', '분자', '조금', '나노', '물질', '조금', '탄소', '분자', '활용'], ['탄소', '원자', '기계', '고리', '체인', '제작', '거나', '분자', '막대', '고리', '하나', '기계', '장치', '구현'], ['프랑스', '출신', '소바', '교수', '탄소', '원자로', '고리', '난이', '초기', '분자', '기계', '개발'], ['영국', '출신', '스토', '더트', '교수', '분자', '막대', '하나', '분자', '고리', '하나', '연결', '로택산', '분자', '기계로', '단계', '발전'], ['네덜란드', '출신', '페링하', '교수', '세계', '가장', '분자', '모터', '개발', '가장', '나노', '자동차', '제작'], ['김기문', '포스텍', '화학', '교수', '로택산', '스위치', '기능', '적도', '한계', '메모리', '소자', '대체', '기술', '활용', '거나', '약물'

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

=> 사전학습 없이 모델을 학습시킬 경우, 사전학습 모델을 사용했을 때와 비교하기 위해 embedding vector size를 200으로 설정하였고, min_count를 1로 하지 않으면 시각화할 때 오류가 나서 1로 지정했습니다. 사전학습 모델의 경우 embedding vector size는 200으로 설정하였고, 마찬가지로 min_count도 1로 설정하였습니다.

In [58]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, min_count = 1)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count # 문장 개수
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 2725


In [59]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 30)

(275586, 288180)

In [60]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

print(model_no_pretrained.wv.most_similar('분자'), "\n", "-"*10)
print(model_no_pretrained.wv.most_similar('질병'), "\n", "-"*10)
print(model_no_pretrained.wv.most_similar('통계'), "\n", "-"*10)
print(model_no_pretrained.wv.most_similar('나라'), "\n", "-"*10)
print(model_no_pretrained.wv.most_similar('화학'), "\n", "-"*10)

[('기계', 0.9999067783355713), ('세계', 0.9998999834060669), ('구현', 0.9998998641967773), ('교수', 0.9998987913131714), ('하나', 0.9998985528945923), ('로봇', 0.9998881816864014), ('헬리콥터', 0.999887228012085), ('드론', 0.9998844861984253), ('제작', 0.9998832941055298), ('실험', 0.9998818635940552)] 
 ----------
[('치료', 0.9997097253799438), ('중국', 0.9997013807296753), ('발생', 0.9997013211250305), ('대한', 0.9996968507766724), ('정확도', 0.9996942281723022), ('방식', 0.9996939897537231), ('차량', 0.9996936917304993), ('진행', 0.9996919631958008), ('미래창조과학부', 0.9996914863586426), ('확인', 0.9996881484985352)] 
 ----------
[('부분', 0.9996874332427979), ('차량', 0.9996817708015442), ('헬리콥터', 0.9996716976165771), ('사용', 0.999667763710022), ('정확도', 0.9996668100357056), ('특징', 0.9996638298034668), ('가지', 0.9996628761291504), ('필요', 0.9996626377105713), ('경우', 0.9996621608734131), ('등장', 0.9996615648269653)] 
 ----------
[('일본', 0.9996799230575562), ('미국', 0.9996359348297119), ('참여', 0.9996358156204224), ('러시아', 0.99962222576141

In [61]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = 200, min_count = 1)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 2725


In [62]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 30708


In [63]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 30)

(288180, 288180)

In [64]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

print(model_pretrained.wv.most_similar('분자'), "\n", "-"*10)
print(model_pretrained.wv.most_similar('질병'), "\n", "-"*10)
print(model_pretrained.wv.most_similar('통계'), "\n", "-"*10)
print(model_pretrained.wv.most_similar('나라'), "\n", "-"*10)
print(model_pretrained.wv.most_similar('화학'), "\n", "-"*10)

[('고분자', 0.770261287689209), ('원자', 0.763414204120636), ('입자', 0.7578503489494324), ('원자핵', 0.7463534474372864), ('물질', 0.7418636083602905), ('탄소', 0.7412014603614807), ('단백질', 0.7245937585830688), ('이중나선', 0.719886839389801), ('염기서열', 0.7186902761459351), ('중합체', 0.7174741625785828)] 
 ----------
[('질환', 0.8036820888519287), ('전염병', 0.7710599303245544), ('합병증', 0.7502058148384094), ('결핵', 0.7414994239807129), ('치매', 0.7294626235961914), ('증상', 0.7264721393585205), ('폐암', 0.7147355079650879), ('고혈압', 0.7097320556640625), ('부작용', 0.7052850723266602), ('식중독', 0.7046220302581787)] 
 ----------
[('통계학', 0.5861766934394836), ('현황', 0.5777990818023682), ('보고서', 0.568411111831665), ('조사', 0.5561819672584534), ('분석', 0.5511950254440308), ('자료', 0.5468671321868896), ('통계적', 0.5429995059967041), ('통계국', 0.5414125919342041), ('편람', 0.5365258455276489), ('서술어', 0.5353242754936218)] 
 ----------
[('제후국', 0.5513360500335693), ('국가', 0.5436817407608032), ('제후', 0.5287867188453674), ('소국', 0.504824697

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

=> 사전 학습 없이 모델을 학습했을 때는 유사한 단어가 잘 출력되지 않았지만, 사전 학습을 한 모델의 경우 vocabulary size가 더 크다보니 사전 학습 없이 모델을 학습했을 때보다 더 정확하고 다양한 단어가 출력된 것 같다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

=> 뭔가 굉장히 잘못된 것 같습니다...

In [65]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

2725

In [66]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [67]:
print(word[:5])
print(vector[:5])

['전적', '그림', '덩어리', '열풍', '개기일식']
[array([-1.4823977 , -1.7022092 ,  0.8513393 ,  0.4843465 ,  0.684738  ,
        0.25724727, -0.29212862,  1.4238727 , -1.0460479 ,  0.23739007,
        1.7984873 , -0.7603374 , -0.4317216 , -0.63984174,  0.6253194 ,
       -0.0623657 ,  1.6323088 ,  0.16190434,  0.1248432 , -1.258012  ,
        0.03637299,  0.9563388 , -0.44245377,  2.6573436 ,  1.3349819 ,
        0.4641754 , -1.9803904 , -0.7445912 ,  0.5287159 , -0.8352456 ,
       -1.4335346 , -0.88082707,  0.33680278, -1.3399051 , -0.23417819,
       -0.23981532,  1.2544274 , -0.9910267 ,  0.49718243, -1.0193526 ,
        1.4253933 ,  0.26024923, -0.31111172,  0.8530443 , -0.08027182,
        0.3694115 ,  0.8264071 , -0.20746484,  0.8198018 ,  0.12235137,
        0.6429065 , -1.454429  ,  0.01791534, -0.20977996, -0.72167563,
       -1.3092346 ,  0.7111904 , -0.01635234, -0.7723865 , -0.6345851 ,
        0.8894887 ,  0.51523066,  0.18600106,  0.31090173,  0.4996418 ,
       -1.106655  , -0.292283

In [68]:
vector[0].shape

(200,)

In [69]:
len(vector)

2725

In [70]:
pca = PCA(random_state = 0).fit_transform(vector)[:,:2]
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:, :2]

In [71]:
print(pca.shape)
pca

(2725, 2)


array([[-0.69592639,  1.58922991],
       [ 2.96558523, -5.31485808],
       [ 1.93459648, -7.28927053],
       ...,
       [-2.8618187 , -0.3853521 ],
       [ 0.94263453,  0.67097622],
       [ 1.34870806,  4.57411942]])

In [72]:
# 시각화해보세요!
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        # color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [73]:
print(tsne.shape)
tsne

(2725, 2)


array([[ 9.2172653e-05,  2.1132597e-05],
       [ 1.1796652e-04,  2.1021071e-04],
       [ 3.2059514e-05, -6.6556240e-05],
       ...,
       [ 4.4554265e-05,  1.7661080e-04],
       [-9.4554685e-05, -5.4687564e-05],
       [ 4.0865743e-05,  8.8457840e-05]], dtype=float32)

In [74]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        # color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [75]:
# t-SNE
tsne1 = TSNE(n_components = 2, random_state = 0, perplexity = 30, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne1[:,0],
    y=tsne1[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        # color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




제임스, 토머스, 크리스, 윌리엄, 피터, 스미스 등 사람 이름이 비슷한 위치에 임베딩됨!

네덜란드, 스위스, 스웨덴, 노르웨이, 프랑스, 러시아 등 국가 이름이 비슷한 위치에 임베딩됨!

산소, 에너지, 입자, 탄소, 수소 등 화학 관련 용어가 비슷한 위치에 임베딩됨!

In [78]:
# t-SNE
tsne1 = TSNE(n_components = 2, random_state = 0, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne1[:,0],
    y=tsne1[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        # color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




흰색, 노란색, 빨간색, 파란색, 초록색, 색깔, 색상 등 색 관련 단어가 비슷한 위치에 임베딩됨!

아들, 사촌, 아버지, 손자 등 가족 관련 단어가 비슷한 위치에 임베딩됨!

# 수고하셨습니다!